In [1]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
#ccba
ccba = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_ccba_full_hashed.csv')
#cdtx
cdtx = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_cdtx0001_full_hashed.csv')
#custinfo
cdtx2 = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_custinfo_full_hashed.csv')
#cdtx
dp = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_dp_full_hashed.csv')
#remit
remit = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_remit1_full_hashed.csv')
#alert_time
alert_time = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_x_alert_date.csv')
#y
y = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\train_y_answer.csv')
#案件名單
doc = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\預測的案件名單及提交檔案範例.csv')

In [3]:
cdtx['country'] = (cdtx['country'] == 130).astype(int)
cdtx['cur_type'] = (cdtx['cur_type'] == 47).astype(int)

In [4]:
cdtx['country'] = cdtx['country'].apply(lambda x: 'TW' if x == 1 else 'FN')
cdtx['cur_type'] = cdtx['cur_type'].apply(lambda x: 'TWD' if x == 1 else 'FND')

In [5]:
doc_merged = cdtx2.merge(doc, on='alert_key', how='right').merge(alert_time, on='alert_key', how='left')
doc_merged = doc_merged[~doc_merged['cust_id'].isnull()]
max_doc = doc_merged.groupby('cust_id')['date'].max().reset_index()
max_doc.rename(columns={'date':'max_date'},inplace=True)
doc_merged = doc_merged.merge(max_doc, on='cust_id', how='left')
prev_list = doc_merged[doc_merged['max_date']>doc_merged['date']]['alert_key'].tolist()

In [6]:
alert_cdtx = cdtx[cdtx['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_cdtx = cdtx[~cdtx['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_cdtx['y'] = 0

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
sar_idx = train_cdtx[train_cdtx['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(y[y['sar_flag']==1]['alert_key'].tolist())]['cust_id'].tolist())].index
train_cdtx.loc[sar_idx, 'y'] = 1

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [8]:
#Data Cleansing
#SAR戶僅留前X天
def sar_pred_d(x, day):
  prev_d = x.groupby('cust_id')['date'].max() - day
  prev_d = prev_d.reset_index()
  prev_d.rename(columns={'date':'prev_d'}, inplace=True)
  x = x.merge(prev_d, on='cust_id', how='left')
  x.drop(x[x['date']<x['prev_d']].index,inplace=True)
  x.pop('prev_d')
  return x

train_cdtx = sar_pred_d(train_cdtx, 30)
alert_cdtx = sar_pred_d(alert_cdtx, 30)


In [9]:
train_cdtx = pd.merge(train_cdtx, cdtx2)
alert_cdtx = pd.merge(alert_cdtx, cdtx2)

In [10]:
class renamer():
    def __init__(self):
        self.d = dict()
    def __call__(self, x):
        if x not in self.d:
            self.d[x] =  0
            return x
        else:
            self.d[x] += 1
            return "%s_%d" % (x, self.d[x])

In [11]:
def total_amt_rolling_3(data):
    total_amt_rolling_3 = (data[data.cur_type == 'TWD'].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(3, on='date')['amt'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_3.columns = ['cust_id', 'alert_key', 'total_amt_rolling_3']
    on_day_total_amt = train_cdtx.groupby(['cust_id', 'date', 'alert_key'])['amt'].sum().reset_index()
    on_day_total_amt.columns = ['cust_id', 'date', 'alert_key', 'on_day_total_amt']
    data = pd.concat([data, total_amt_rolling_3], axis=1)
    data = data.rename(columns=renamer())  
    data = pd.merge(data, on_day_total_amt, how='left') 
    data['total_amt_rolling_3'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_3']) else x['total_amt_rolling_3'], axis=1)
    return data

train_cdtx = total_amt_rolling_3(train_cdtx)
alert_cdtx = total_amt_rolling_3(alert_cdtx)

In [12]:
def total_amt_rolling_7(data):
    total_amt_rolling_7 = (data[data.cur_type == 'TWD'].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(7, on='date')['amt'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_7.columns = ['cust_id', 'alert_key', 'total_amt_rolling_7']
    data = pd.concat([data, total_amt_rolling_7], axis=1)
    data = data.rename(columns=renamer())
    data['total_amt_rolling_7'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_7']) else x['total_amt_rolling_7'], axis=1)
    return data
train_cdtx = total_amt_rolling_7(train_cdtx)
alert_cdtx = total_amt_rolling_7(alert_cdtx)

In [13]:
def total_amt_rolling_14(data):
    total_amt_rolling_14 = (data[data.cur_type == 'TWD'].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(14, on='date')['amt'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_14.columns = ['cust_id', 'alert_key', 'total_amt_rolling_14']
    data = pd.concat([data, total_amt_rolling_14], axis=1)
    data = data.rename(columns=renamer())
    data['total_amt_rolling_14'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_14']) else x['total_amt_rolling_14'], axis=1)
    return data
train_cdtx = total_amt_rolling_14(train_cdtx)
alert_cdtx = total_amt_rolling_14(alert_cdtx)

In [14]:
def amt_diff_func(data):
  session_amt_diff = data.groupby(['cust_id','country'])['amt'].count().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='country', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['country_amt_diff_ratio'] = \
    abs(session_amt_diff['TW'] - session_amt_diff['FN']) / abs(session_amt_diff['TW'] + session_amt_diff['FN'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','country_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')
  

  session_amt_diff = data.groupby(['cust_id','cur_type'])['amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='cur_type', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['cur_amt_diff_ratio'] = \
    session_amt_diff['TWD'] / abs(session_amt_diff['TWD'] + session_amt_diff['FND'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','cur_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')

  return data

In [15]:
train_cdtx = amt_diff_func(train_cdtx)
alert_cdtx = amt_diff_func(alert_cdtx)

In [16]:
def amt_diff_func(data):
  session_amt_diff = data.groupby(['cust_id','country'])['amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='country', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['country_amt_diff_ratio'] = \
    abs(session_amt_diff['TW'] - session_amt_diff['FN']) / abs(session_amt_diff['TW'] + session_amt_diff['FN'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','country_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')
  

  session_amt_diff = data.groupby(['cust_id','cur_type'])['amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='cur_type', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['cur_amt_diff_ratio'] = \
    abs(session_amt_diff['TWD'] - session_amt_diff['FND']) / abs(session_amt_diff['TWD'] + session_amt_diff['FND'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','cur_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')

  return data

In [17]:
train_cdtx = amt_diff_func(train_cdtx)
alert_cdtx = amt_diff_func(alert_cdtx)

In [18]:
train_cdtx['country'] = train_cdtx['country'].astype('category')
train_cdtx['cur_type'] = train_cdtx['cur_type'].astype('category')

alert_cdtx['country'] = alert_cdtx['country'].astype('category')
alert_cdtx['cur_type'] = alert_cdtx['cur_type'].astype('category')

train_cdtx['RISK_RANK'] = train_cdtx['RISK_RANK'].astype('category')
train_cdtx['OCCUPATION_CODE'] = train_cdtx['OCCUPATION_CODE'].astype('category')
train_cdtx['AGE'] = train_cdtx['AGE'].astype('category')
alert_cdtx['RISK_RANK'] = alert_cdtx['RISK_RANK'].astype('category')
alert_cdtx['OCCUPATION_CODE'] = alert_cdtx['OCCUPATION_CODE'].astype('category')
alert_cdtx['AGE'] = alert_cdtx['AGE'].astype('category')

In [22]:
cat_cols = ['cust_id', 'date', 'RISK_RANK', 'OCCUPATION_CODE', 'AGE']
train_cdtx = train_cdtx.sort_values(['cust_id', 'date'])
train_cdtx['RISK_RANK'] = train_cdtx['RISK_RANK'].ffill()
train_cdtx['OCCUPATION_CODE'] = train_cdtx['OCCUPATION_CODE'].ffill()
train_cdtx['AGE'] = train_cdtx['AGE'].ffill()

In [24]:
train_cdtx

,cust_id,date,country,cur_type,amt,y,alert_key,RISK_RANK,OCCUPATION_CODE,TOTAL_ASSET,...,cust_id_1_1,alert_key_1_1,total_amt_rolling_7,cust_id_1,alert_key_1,total_amt_rolling_14,country_amt_diff_ratio_x,cur_amt_diff_ratio_x,country_amt_diff_ratio_y,cur_amt_diff_ratio_y
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,372,TW,TWD,237.0,0,195913,1,17.0,32758,...,0015cc01f553512007705edf855baa16f1b57a16fb107e...,195913.0,237.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,195913.0,237.0,0.333333,0.998462,0.996923,0.996923
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,386,TW,TWD,412.0,0,195913,1,17.0,32758,...,0015cc01f553512007705edf855baa16f1b57a16fb107e...,195913.0,412.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,195913.0,412.0,0.333333,0.998462,0.996923,0.996923
2,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,0,269235,3,4.0,629787,...,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,269235.0,22874.0,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,269235.0,22874.0,0.953488,0.999996,0.999991,0.999991
3,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,0,270967,3,4.0,629787,...,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,269235.0,22874.0,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,269235.0,22874.0,0.953488,0.999996,0.999991,0.999991
4,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,0,271556,3,4.0,629787,...,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,269235.0,22874.0,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,269235.0,22874.0,0.953488,0.999996,0.999991,0.999991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,359,TW,TWD,229.0,0,303894,1,14.0,458455,...,NaN,NaN,229.0,NaN,NaN,229.0,0.916667,0.999975,0.976025,0.999951
378485,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,360,TW,TWD,1112.0,0,303894,1,14.0,458455,...,NaN,NaN,1112.0,NaN,NaN,1112.0,0.916667,0.999975,0.976025,0.999951
378486,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,361,TW,TWD,59.0,0,303894,1,14.0,458455,...,NaN,NaN,59.0,NaN,NaN,59.0,0.916667,0.999975,0.976025,0.999951
378487,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,364,TW,TWD,156.0,0,303894,1,14.0,458455,...,NaN,NaN,156.0,NaN,NaN,156.0,0.916667,0.999975,0.976025,0.999951


In [25]:
train_cdtx['ratio_total_aum_7'] = train_cdtx.total_amt_rolling_7 /train_cdtx.TOTAL_ASSET
train_cdtx['ratio_total_aum_1'] = train_cdtx.on_day_total_amt  /train_cdtx.TOTAL_ASSET 
train_cdtx['ratio_total_aum_14'] = train_cdtx.total_amt_rolling_14/train_cdtx.TOTAL_ASSET
train_cdtx['ratio_total_aum_3'] = train_cdtx.total_amt_rolling_3  /train_cdtx.TOTAL_ASSET
alert_cdtx['ratio_total_aum_7'] = alert_cdtx.total_amt_rolling_7 /alert_cdtx.TOTAL_ASSET
alert_cdtx['ratio_total_aum_1'] = alert_cdtx.on_day_total_amt  /alert_cdtx.TOTAL_ASSET 
alert_cdtx['ratio_total_aum_14'] = alert_cdtx.total_amt_rolling_14/alert_cdtx.TOTAL_ASSET
alert_cdtx['ratio_total_aum_3'] = alert_cdtx.total_amt_rolling_3  /alert_cdtx.TOTAL_ASSET

In [26]:
alert_cdtx.replace([np.inf, -np.inf], 0, inplace=True)

In [27]:
train_cdtx.columns

Index(['cust_id', 'date', 'country', 'cur_type', 'amt', 'y', 'alert_key',
       'RISK_RANK', 'OCCUPATION_CODE', 'TOTAL_ASSET', 'AGE', 'cust_id_1',
       'alert_key_1', 'total_amt_rolling_3', 'on_day_total_amt', 'cust_id_1_1',
       'alert_key_1_1', 'total_amt_rolling_7', 'cust_id_1', 'alert_key_1',
       'total_amt_rolling_14', 'country_amt_diff_ratio_x',
       'cur_amt_diff_ratio_x', 'country_amt_diff_ratio_y',
       'cur_amt_diff_ratio_y', 'ratio_total_aum_7', 'ratio_total_aum_1',
       'ratio_total_aum_14', 'ratio_total_aum_3'],
      dtype='object')

In [29]:
train_cdtx.loc[:, ~train_cdtx.columns.isin(cat_cols+['country','cur_type'])].fillna(0,inplace=True)
alert_cdtx.loc[:, ~alert_cdtx.columns.isin(cat_cols+['country','cur_type'])].fillna(0,inplace=True)

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [47]:
train_col = ['cust_id', 'date', 'amt'
             , 'country', 'cur_type',
       'RISK_RANK', 'OCCUPATION_CODE'
             , 'TOTAL_ASSET'
             , 'AGE'
             ,'total_amt_rolling_3', 'on_day_total_amt',
        'total_amt_rolling_7',
       'total_amt_rolling_14'
       , 'ratio_total_aum_7', 'ratio_total_aum_1',
       'ratio_total_aum_14', 'ratio_total_aum_3'
       , 'y']
x_col = train_col[3:-1]
train_data = train_cdtx[train_col]
train_data

,cust_id,date,amt,country,cur_type,RISK_RANK,OCCUPATION_CODE,TOTAL_ASSET,AGE,total_amt_rolling_3,on_day_total_amt,total_amt_rolling_7,total_amt_rolling_14,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3,y
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,372,237.0,TW,TWD,1,17.0,32758,3,237.0,237.0,237.0,237.0,0.007235,0.007235,0.007235,0.007235,0
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,386,412.0,TW,TWD,1,17.0,32758,3,412.0,412.0,412.0,412.0,0.012577,0.012577,0.012577,0.012577,0
2,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,508.0,TW,TWD,3,4.0,629787,3,22874.0,22874.0,22874.0,22874.0,0.036320,0.036320,0.036320,0.036320,0
3,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,508.0,TW,TWD,3,4.0,629787,3,22874.0,22874.0,22874.0,22874.0,0.036320,0.036320,0.036320,0.036320,0
4,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,508.0,TW,TWD,3,4.0,629787,3,22995.0,22874.0,22874.0,22874.0,0.036320,0.036320,0.036320,0.036512,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,359,229.0,TW,TWD,1,14.0,458455,5,229.0,229.0,229.0,229.0,0.000500,0.000500,0.000500,0.000500,0
378485,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,360,1112.0,TW,TWD,1,14.0,458455,5,1112.0,1112.0,1112.0,1112.0,0.002426,0.002426,0.002426,0.002426,0
378486,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,361,59.0,TW,TWD,1,14.0,458455,5,59.0,59.0,59.0,59.0,0.000129,0.000129,0.000129,0.000129,0
378487,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,364,156.0,TW,TWD,1,14.0,458455,5,156.0,156.0,156.0,156.0,0.000340,0.000340,0.000340,0.000340,0


In [48]:
train_data = train_data.drop_duplicates()

In [49]:
Y = train_data['y']
X = train_data[x_col]

In [50]:
X

,country,cur_type,RISK_RANK,OCCUPATION_CODE,TOTAL_ASSET,AGE,total_amt_rolling_3,on_day_total_amt,total_amt_rolling_7,total_amt_rolling_14,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3
0,TW,TWD,1,17.0,32758,3,237.0,237.0,237.0,237.0,0.007235,0.007235,0.007235,0.007235
1,TW,TWD,1,17.0,32758,3,412.0,412.0,412.0,412.0,0.012577,0.012577,0.012577,0.012577
2,TW,TWD,3,4.0,629787,3,22874.0,22874.0,22874.0,22874.0,0.036320,0.036320,0.036320,0.036320
4,TW,TWD,3,4.0,629787,3,22995.0,22874.0,22874.0,22874.0,0.036320,0.036320,0.036320,0.036512
8,TW,TWD,3,4.0,629787,3,238.0,22874.0,24453.0,22874.0,0.038827,0.036320,0.036320,0.000378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,TW,TWD,1,14.0,458455,5,229.0,229.0,229.0,229.0,0.000500,0.000500,0.000500,0.000500
378485,TW,TWD,1,14.0,458455,5,1112.0,1112.0,1112.0,1112.0,0.002426,0.002426,0.002426,0.002426
378486,TW,TWD,1,14.0,458455,5,59.0,59.0,59.0,59.0,0.000129,0.000129,0.000129,0.000129
378487,TW,TWD,1,14.0,458455,5,156.0,156.0,156.0,156.0,0.000340,0.000340,0.000340,0.000340


In [51]:
X.replace([np.inf, -np.inf], 0, inplace=True)

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [52]:
X.apply(lambda x: np.isinf(x).values.sum())

TypeError: Object with dtype category cannot perform the numpy op isinf

In [60]:
#model training 1
#逐筆交易處理
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0,
        enable_categorical=True,
        use_label_encoder=False,
        predictor= "gpu_predictor")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#previous 30days label of 1
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 73.76%
#0.9921875
#0.5869618097786427

F1 score: 99.59%
0.9985417146365799
0.9933572573871879


In [61]:
pd.DataFrame({'columns':list(X.columns),'score':model.feature_importances_})

,columns,score
0,country,0.005765
1,cur_type,0.932563
2,RISK_RANK,0.006450
3,OCCUPATION_CODE,0.026851
4,TOTAL_ASSET,0.007372
5,AGE,0.007456
6,total_amt_rolling_3,0.001206
7,on_day_total_amt,0.005952
8,total_amt_rolling_7,0.000711
9,total_amt_rolling_14,0.000913


In [62]:
#彙整model training 1偵測結果
#predict_proba以0.05機率區間為一個欄位判斷分佈
def mapping(proba):
    if proba <= 0.05:
        return 1
    elif 0.05 < proba <= 0.1:
        return 2
    elif 0.1 < proba <= 0.15:
        return 3
    elif 0.15 < proba <= 0.2:
        return 4
    elif 0.2 < proba <= 0.25:
        return 5
    elif 0.25 < proba <= 0.3:
        return 6
    elif 0.3 < proba <= 0.35:
        return 7
    elif 0.35 < proba <= 0.4:
        return 8
    elif 0.4 < proba <= 0.45:
        return 9
    elif 0.45 < proba <= 0.5:
        return 10
    elif 0.5 < proba <= 0.55:
        return 11
    elif 0.55 < proba <= 0.6:
        return 12
    elif 0.6 < proba <= 0.65:
        return 13
    elif 0.65 < proba <= 0.7:
        return 14
    elif 0.7 < proba <= 0.75:
        return 15
    elif 0.75 < proba <= 0.8:
        return 16
    elif 0.8 < proba <= 0.85:
        return 17
    elif 0.85 < proba <= 0.9:
        return 18
    elif 0.9 < proba <= 0.95:
        return 19
    elif 0.95 < proba <= 1:
        return 20

In [63]:
train_data['proba'] = model.predict_proba(X)[:,1]

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
result = train_data[['cust_id','proba','y']]
result['level'] = result["proba"].map(mapping)
result = result[['cust_id', 'level', 'y']]
result = result.groupby(['cust_id','level']).count().reset_index().pivot_table(index='cust_id', columns='level', values='y')
result.fillna(0, inplace=True)
result = result.div(result.sum(axis=1), axis=0)
result = result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(result.columns[1:])):
  result[col] = 0.0
result = result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
result = result.merge(train_data[['cust_id','y']].drop_duplicates(), on='cust_id', how='left')
result.pop('cust_id')

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0       0015cc01f553512007705edf855baa16f1b57a16fb107e...
1       002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...
2       002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...
3       0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...
4       0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...
                              ...                        
3516    ffa5bcc6f88df5c33e809da4daa54de0031803f37d8f11...
3517    ffae7fef923c78298cd647a3d573e48cc1404b07ac794a...
3518    ffdbc1a5135cb6feddf7355e5656abf6973f21639c7b9b...
3519    ffdd3f98b841d4d0ef58ad0d63abddac83ffb5babf973c...
3520    ffeb846f2294ec4397f800736dc1e14eb754da5425398d...
Name: cust_id, Length: 3521, dtype: object

In [65]:
#model training 2
#歸戶判斷是否報SAR
result_col = list(result.columns)
result_col.remove('y')
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(result[result_col], result['y'], test_size=test_size, random_state=seed)
model_2nd = XGBClassifier()
model_2nd.fit(X_train, y_train)
y_pred = model_2nd.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 36.36%
#0.7142857142857143
#0.24390243902439024

F1 score: 74.42%
0.8421052631578947
0.6666666666666666


In [66]:
#Submission
alert_x = alert_cdtx[x_col]
alert_data = alert_cdtx
alert_data['proba'] = model.predict_proba(alert_x)[:,1]
alert_result = alert_data[['cust_id','proba']]
alert_result['level'] = alert_result["proba"].map(mapping)
alert_result = alert_result[['cust_id', 'level']]
alert_result['cnt'] = 1
alert_result = alert_result.groupby(['cust_id','level'])['cnt'].count().reset_index().pivot_table(index='cust_id', columns='level', values='cnt')
alert_result.fillna(0, inplace=True)
alert_result = alert_result.div(alert_result.sum(axis=1), axis=0)
alert_result = alert_result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(alert_result.columns[1:])):
  alert_result[col] = 0.0
alert_result = alert_result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
#alert_result.pop('cust_id')
alert_pred = model_2nd.predict_proba(alert_result[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]])
# evaluate predictions
final = alert_result[['cust_id']]
final['probability'] = alert_pred[:,1]
final

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


level,cust_id,probability
0,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,0.003187
1,01c705b457297805c1c08137cb3adb164b8f393c265db2...,0.003187
2,02e2cb9bb3a904b4e44b7e40f54d536ca5941db440ba00...,0.003187
3,035e2ebcd5c23f4dd0f1fc81a1fca8afd3121f7431bb58...,0.003187
4,03d4cbfcb065501bebb2b75ea4df2b779df0192a183af3...,0.003187
...,...,...
419,fec476c1d12e20806dc384748798a07bfd972ed0626890...,0.003187
420,fee6ee67b6db61c5a7788074e412b0c0ac77b3762b3b98...,0.003187
421,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,0.003187
422,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,0.003187


In [ ]:
final = alert_time.merge(cdtx2[['alert_key', 'cust_id']].merge(final, on='cust_id'), on='alert_key', how='left')[['alert_key', 'probability']]

In [ ]:
final.fillna(0, inplace=True)
final

In [ ]:
doc = doc[['alert_key']]
doc

In [ ]:
final = doc.merge(final, on='alert_key', how='left')
final.fillna(0,inplace=True)
final

In [ ]:
final.loc[final[final['alert_key'].isin(prev_list)].index,'probability']=0

In [ ]:
final.to_csv('/content/drive/MyDrive/訓練資料集_first/submission.csv')

In [67]:
final[final['probability']>0]

level,cust_id,probability
0,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,0.003187
1,01c705b457297805c1c08137cb3adb164b8f393c265db2...,0.003187
2,02e2cb9bb3a904b4e44b7e40f54d536ca5941db440ba00...,0.003187
3,035e2ebcd5c23f4dd0f1fc81a1fca8afd3121f7431bb58...,0.003187
4,03d4cbfcb065501bebb2b75ea4df2b779df0192a183af3...,0.003187
...,...,...
419,fec476c1d12e20806dc384748798a07bfd972ed0626890...,0.003187
420,fee6ee67b6db61c5a7788074e412b0c0ac77b3762b3b98...,0.003187
421,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,0.003187
422,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,0.003187
